# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp


In [14]:
from astropy.time import Time

from jorbit.utils.horizons import horizons_bulk_vector_query

t0 = Time("2020-01-01")

# for _i in range(20):
vecs = horizons_bulk_vector_query(
    # target="K12TB3C",
    target="K12Tb3C",
    center="500@0",
    times=t0,
    disable_astroquery=True,
)
x0 = jnp.array([vecs["x"], vecs["y"], vecs["z"]]).T[0]
v0 = jnp.array([vecs["vx"], vecs["vy"], vecs["vz"]]).T[0]

x0

Array([-2.53956678,  2.37614377,  1.00774371], dtype=float64)

Array([-1.6913443 , -3.01939326, -1.04277701], dtype=float64)